In [2]:
import numpy as np
import math
from sklearn.metrics import classification_report

## Micro/Macro Avg, percision, recall from Confusion Matrix

In [4]:
cfm = np.array([[28, 5, 7],
                [10, 10, 0],
                [0, 8, 12]
                ])
tn = ['acro','base','claw']

l = len(cfm)
ypred = np.array([])
ytrue = np.array([])
# Creating dummy ytrue, ypred for comparison. sklearn will do the rest of the job
for i in range(l):
    for j in range(l):
        tmp = np.ones([cfm[i,j]])
        ytrue = np.append(ytrue,i*tmp, axis = 0)
        ypred = np.append(ypred,j*tmp, axis = 0)

cfr = classification_report(ytrue, ypred,target_names=tn)
print(cfr)

              precision    recall  f1-score   support

        acro       0.74      0.70      0.72        40
        base       0.43      0.50      0.47        20
        claw       0.63      0.60      0.62        20

    accuracy                           0.62        80
   macro avg       0.60      0.60      0.60        80
weighted avg       0.64      0.62      0.63        80



In [3]:
def describe(i_vals, tn, cfm, beta=1):
    l = len(tn)
    for i in i_vals:
        p = cfm[i,i]/np.sum(cfm[:,i])
        print(f'Precision({tn[i]}) = {cfm[i,i]}/(' + '+'.join([f'{cfm[j,i]}' for j in range(l)]) + f') = {p:.4g}')
        r = cfm[i,i]/np.sum(cfm[i,:])
        print(f'Recall({tn[i]}) = {cfm[i,i]}/(' + '+'.join([f'{cfm[i,j]}' for j in range(l)]) + f') = {r:.4g}')
        f = (1+beta**2)*p*r / (beta**2*p + r)
        print(f'f_{beta}({tn[i]}) = (1+{beta**2})*{p:.4g}*{r:.4g} / ({beta**2}*{p:.4g} + {r:.4g}) = {f:.4g}')
        print(f'Sensitivity({tn[i]}) = {cfm[i,i]}/(' + '+'.join([f'{cfm[i,j]}' for j in range(l)]) + f') = {r:.4g}')
        TN = np.sum(cfm)+cfm[i,i]-np.sum(cfm[:,i])-np.sum(cfm[i,:])
        total_neg = np.sum(cfm)-np.sum(cfm[i,:])
        sp = TN/total_neg
        print(f'Specificity({tn[i]}) = {TN}/(' + '+'.join([f'{np.sum(cfm[:,j])-cfm[i,j]}' for j in range(l)]) + f') = {sp:.4g}')
        print('')

#cfrd = classification_report(ytrue, ypred,target_names=tn, output_dict=True)
#cfrd['bird']['recall']
def avr(tn, cfm, beta=1):
    l = len(tn)
    tp = np.array([])
    tpfp = np.array([])
    tpfn = np.array([])
    p = np.array([])
    r = np.array([])
    f = np.array([])
    for i in range(l):
        pe = cfm[i,i]/np.sum(cfm[:,i])
        re = cfm[i,i]/np.sum(cfm[i,:])

        tp = np.append(tp,[cfm[i,i]],axis=0)
        tpfp = np.append(tpfp,[np.sum(cfm[:,i])],axis=0)
        tpfn = np.append(tpfn,[np.sum(cfm[i,:])],axis=0)
        p = np.append(p,[pe],axis=0)
        r = np.append(r,[re],axis=0)

    pM = np.sum(p)/l
    rM = np.sum(r)/l
    print(f'Percision_M = (' + '+'.join([f'{pe:.4g}' for pe in p]) + f')/{l} = {pM:.4g}')
    print(f'Recall_M = (' + '+'.join([f'{re:.4g}' for re in r]) + f')/{l} = {rM:.4g}')
    fM = (1+beta**2)*pM*rM / (beta**2*pM + rM)
    print(f'f_{beta}M = (1+{beta**2})*{pM:.4g}*{rM:.4g} / ({beta**2}*{pM:.4g} + {rM:.4g}) = {fM:.4g}')
    print('')

    pmu = np.sum(tp)/np.sum(tpfp)
    rmu = np.sum(tp)/np.sum(tpfn)
    print(f'Percision_mu = (' + '+'.join([f'{t:.4g}' for t in tp]) + f') / ('+'+'.join([f'{t:.4g}' for t in tpfp]) + f') = {pmu:.4g}')
    print(f'Recall_mu = (' + '+'.join([f'{t:.4g}' for t in tp]) + f') / ('+'+'.join([f'{t:.4g}' for t in tpfn]) + f') = {rmu:.4g}')
    fmu = (1+beta**2)*pmu*rmu / (beta**2*pmu + rmu)
    print(f'f_{beta}mu = (1+{beta**2})*{pmu:.4g}*{rmu:.4g} / ({beta**2}*{pmu:.4g} + {rmu:.4g}) = {fmu:.4g}')
    print('')
    
    w = np.array([np.sum(cfm[i,:])/np.sum(cfm) for i in range(l)])
    pw = np.sum([p[i]*w[i] for i in range(l)])
    rw = np.sum([r[i]*w[i] for i in range(l)])
    print(f'Percision_w = ' + ' + '.join([f'{p[i]:.4g}*{w[i]:.4g}' for i in range(l)]) + f'  = {pw:.4g}')
    print(f'Recall_w = ' + ' + '.join([f'{r[i]:.4g}*{w[i]:.4g}' for i in range(l)]) + f' = {rw:.4g}')
    fw = (1+beta**2)*pw*rw / (beta**2*pw + rw)
    print(f'f_{beta}w = (1+{beta**2})*{pw:.4g}*{rw:.4g} / ({beta**2}*{pw:.4g} + {rw:.4g}) = {fw:.4g}')
    #print('')

In [7]:
describe([0,1,2],tn,cfm)

Precision(acro) = 28/(28+10+0) = 0.7368
Recall(acro) = 28/(28+5+7) = 0.7
f_1(acro) = (1+1)*0.7368*0.7 / (1*0.7368 + 0.7) = 0.7179
Sensitivity(acro) = 28/(28+5+7) = 0.7
Specificity(acro) = 30/(10+18+12) = 0.75

Precision(base) = 10/(5+10+8) = 0.4348
Recall(base) = 10/(10+10+0) = 0.5
f_1(base) = (1+1)*0.4348*0.5 / (1*0.4348 + 0.5) = 0.4651
Sensitivity(base) = 10/(10+10+0) = 0.5
Specificity(base) = 47/(28+13+19) = 0.7833

Precision(claw) = 12/(7+0+12) = 0.6316
Recall(claw) = 12/(0+8+12) = 0.6
f_1(claw) = (1+1)*0.6316*0.6 / (1*0.6316 + 0.6) = 0.6154
Sensitivity(claw) = 12/(0+8+12) = 0.6
Specificity(claw) = 53/(38+15+7) = 0.8833



In [8]:
avr(tn,cfm)

Percision_M = (0.7368+0.4348+0.6316)/3 = 0.6011
Recall_M = (0.7+0.5+0.6)/3 = 0.6
f_1M = (1+1)*0.6011*0.6 / (1*0.6011 + 0.6) = 0.6005

Percision_mu = (28+10+12) / (38+23+19) = 0.625
Recall_mu = (28+10+12) / (40+20+20) = 0.625
f_1mu = (1+1)*0.625*0.625 / (1*0.625 + 0.625) = 0.625

Percision_w = 0.7368*0.5 + 0.4348*0.25 + 0.6316*0.25  = 0.635
Recall_w = 0.7*0.5 + 0.5*0.25 + 0.6*0.25 = 0.625
f_1w = (1+1)*0.635*0.625 / (1*0.635 + 0.625) = 0.63


## Bayes Rule

In [3]:
# Calculating P(A|B)

#P(A) - Prior
pa = 0.22
#P(B) - Prior of evidence
pb = 0.2
#P(B|A) - Posterior
pba = 0.02

pab = pba*pa/pb
print(pb)

0.2


## Entropy

In [4]:
def entropy(ps, i = ''):
    print('H'+i+' = ' + ' + '.join([f'-{p}*log2({p})' for p in ps]))
    es = [0 if not p else -p*math.log2(p) for p in ps]
    entr = np.sum(es)
    print('= ' + ' + '.join([f'{e:.4g}' for e in es]) + f' = {entr:.4g}')
    return entr

In [7]:
ps = [0.6,0.4]
entropy(ps)

H = -0.6*log2(0.6) + -0.4*log2(0.4)
= 0.4422 + 0.5288 = 0.971


0.9709505944546686

## Normal/Gaussian Distribution

In [6]:
"""
Calculates the probability that event X occurs based on probability density function
"""
pdf = lambda mu, sigma, x: math.exp(-((x-mu)/sigma)**2/2)/(sigma*2*math.pi)

pdf(1,3,4)

0.03217745087668464